<a href="https://colab.research.google.com/github/Jacyndy/LLm_project/blob/main/notebooks/Preprocess%2C_tokenize_and_DocRep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
# import the necessary libraries
import string
import pandas as pd
from datasets import load_dataset

In [ ]:
# load the daataset
ds = load_dataset('imdb')
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
# view the first part of the data
ds['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
# select a small subset of the data
small_train_dataset = ds["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = ds["test"].shuffle(seed=42).select(range(1000))

In [ ]:
# convert to a dataframe
ds_train = pd.DataFrame(small_train_dataset)

In [ ]:
# view rhe first few rows
ds_train.head()

,text,label
0,There is no relation at all between Fortier an...,1
1,This movie is a great. The plot is very true t...,1
2,"George P. Cosmatos' ""Rambo: First Blood Part I...",0
3,In the process of trying to establish the audi...,1
4,"Yeh, I know -- you're quivering with excitemen...",0


In [ ]:
ds_test = pd.DataFrame(small_eval_dataset)

Grab a single review.

In [ ]:
text = ds_train['text'][0]
print(text)

There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier's plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it's the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...


## Removing punctuation
Review the available punctuation characters.

In [ ]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


Create a function to remove punctuation.

In [ ]:
def remove_punctuation(text):
    text = "".join([char for char in text if char not in string.punctuation])

    return text

Trying the function out on a single piece of text.

In [ ]:
text_no_punct = remove_punctuation(text)
print(text_no_punct)

There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes Profiler looks crispy Fortier looks classic Profiler plots are quite simple Fortiers plot are far more complicated Fortier looks more like Prime Suspect if we have to spot similarities The main character is weak and weirdo but have clairvoyance People like to compare to judge to evaluate How about just enjoying Funny thing too people writing Fortier looks American but on the other hand arguing they prefer American series  Maybe its the language or the spirit but I think this series is more English than American By the way the actors are really good and funny The acting is not superficial at all


Apply function to remove punctuation.

In [ ]:
# for the train data
ds_train['text_no_punct'] = ds_train['text'].apply(lambda x: remove_punctuation(x))

In [ ]:
ds_train.head()

,text,label,text_no_punct
0,There is no relation at all between Fortier an...,1,There is no relation at all between Fortier an...
1,This movie is a great. The plot is very true t...,1,This movie is a great The plot is very true to...
2,"George P. Cosmatos' ""Rambo: First Blood Part I...",0,George P Cosmatos Rambo First Blood Part II is...
3,In the process of trying to establish the audi...,1,In the process of trying to establish the audi...
4,"Yeh, I know -- you're quivering with excitemen...",0,Yeh I know youre quivering with excitement We...


In [ ]:
# for the test data
ds_test['text_no_punct'] = ds_test['text'].apply(lambda x: remove_punctuation(x))

## Tokenization

Define a function to first make all the letters lowercase and then split each word into a separate string (giving us a list of strings). Note that the splitting happens based on spaces.

In [ ]:
def tokenize(text):
    tokens = text.lower().split()

    return tokens

Now, let's apply that to each review (with punctuation removed).

In [ ]:
ds_train['tokens'] = ds_train['text_no_punct'].apply(lambda x: tokenize(x))

In [ ]:
ds_train.head()

,text,label,text_no_punct,tokens
0,There is no relation at all between Fortier an...,1,There is no relation at all between Fortier an...,"[there, is, no, relation, at, all, between, fo..."
1,This movie is a great. The plot is very true t...,1,This movie is a great The plot is very true to...,"[this, movie, is, a, great, the, plot, is, ver..."
2,"George P. Cosmatos' ""Rambo: First Blood Part I...",0,George P Cosmatos Rambo First Blood Part II is...,"[george, p, cosmatos, rambo, first, blood, par..."
3,In the process of trying to establish the audi...,1,In the process of trying to establish the audi...,"[in, the, process, of, trying, to, establish, ..."
4,"Yeh, I know -- you're quivering with excitemen...",0,Yeh I know youre quivering with excitement We...,"[yeh, i, know, youre, quivering, with, excitem..."


In [ ]:
ds_test['tokens'] = ds_test['text_no_punct'].apply(lambda x: tokenize(x))

## Removing Stop Words

First, import the stopwords corpus from NLTK.

In [ ]:
 import nltk
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

Make a function to remove stop words:

In [ ]:
def remove_stopwords(text):
    text = [word for word in text if word not in stop_words]

    return text

Now, let's apply that to each review (after tokenization with punctuation removed).

In [ ]:
# for train data
ds_train['tokens_nostop'] = ds_train['tokens'].apply(lambda x: remove_stopwords(x))

In [ ]:
ds_train.head()

,text,label,text_no_punct,tokens,tokens_nostop
0,There is no relation at all between Fortier an...,1,There is no relation at all between Fortier an...,"[there, is, no, relation, at, all, between, fo...","[relation, fortier, profiler, fact, police, se..."
1,This movie is a great. The plot is very true t...,1,This movie is a great The plot is very true to...,"[this, movie, is, a, great, the, plot, is, ver...","[movie, great, plot, true, book, classic, writ..."
2,"George P. Cosmatos' ""Rambo: First Blood Part I...",0,George P Cosmatos Rambo First Blood Part II is...,"[george, p, cosmatos, rambo, first, blood, par...","[george, p, cosmatos, rambo, first, blood, par..."
3,In the process of trying to establish the audi...,1,In the process of trying to establish the audi...,"[in, the, process, of, trying, to, establish, ...","[process, trying, establish, audiences, empath..."
4,"Yeh, I know -- you're quivering with excitemen...",0,Yeh I know youre quivering with excitement We...,"[yeh, i, know, youre, quivering, with, excitem...","[yeh, know, youre, quivering, excitement, well..."


In [ ]:
# for the test
ds_test['tokens_nostop'] = ds_test['tokens'].apply(lambda x: remove_stopwords(x))

# Document Representation

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from datasets import load_dataset

In [ ]:
# for train data
ds_train['tokens_nostop_string'] = ds_train['tokens_nostop'].astype(str)

In [ ]:
# for test data
ds_test['tokens_nostop_string'] = ds_test['tokens_nostop'].astype(str)

In [ ]:
# 1. Bag of Words (BoW)
vectorizer = CountVectorizer(max_features=5000)
ds_train_bow = vectorizer.fit_transform(ds_train['tokens_nostop_string'])
ds_test_bow = vectorizer.transform(ds_test['tokens_nostop_string'])

In [ ]:
ds_train_bow

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 80257 stored elements and shape (1000, 5000)>

# Modelling

In [ ]:
# import the necessary libraries, split the data into training and testing and fit the model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


# Split data into train and test sets
X_train = ds_train_bow
X_test = ds_test_bow
y_train = ds_train['label']
y_test = ds_test['label']

# Train a classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

# Evaluate the classifier
accuracy_1 = clf.score(X_test, y_test)
print("Accuracy:", accuracy_1)

# Evaluate model performance
accuracy_2 = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy_2:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.775
Model Accuracy: 0.7750
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.77      0.78       512
           1       0.76      0.78      0.77       488

    accuracy                           0.78      1000
   macro avg       0.77      0.78      0.77      1000
weighted avg       0.78      0.78      0.78      1000

